In [2]:
import numpy as np
import pandas as pd
from faker import Faker
import random
import matplotlib.pyplot as plt
import secrets
import string
import psycopg2
from datetime import timedelta, datetime

# Fake client data

In [3]:
def getRandomPasswordString():
    alphabet = string.ascii_letters + string.digits + string.punctuation.replace(',', '')  # Exclure la virgule

    password = secrets.choice(string.ascii_uppercase) + \
               secrets.choice(string.ascii_lowercase) + \
               secrets.choice(string.digits) + \
               secrets.choice(string.punctuation.replace(',', '')) + \
               ''.join(secrets.choice(alphabet) for i in range(12))

    return password


In [4]:
getRandomPasswordString()

'Jx0\\6t32Gk.413@='

In [73]:
def generate_data():
    fake = Faker()
    liste_des_domaines = ['com','net','org','gov','fr']

    list_rows = []
    nb_row = random.randint(10000, 12000)

    generated_ids = set()   # Utilisation d'un ensemble pour stocker les IDs générés

    for i in range(nb_row):
        first_name = fake.first_name()
        last_name = fake.last_name()
        company = fake.company().split()[0].strip(',')
        dns_org = fake.random_choices(elements=liste_des_domaines, length=1)[0]
        email = f"{first_name}.{last_name}@{company}.{dns_org}".lower()
        unique_id = random.randint(100000, 999999)
        pwd = getRandomPasswordString()
        condition = True
        while (condition):
            age = int(np.random.gamma(9, 5)) - 20
            if ((age > 15) and (age < 66)):
                condition = False

        if unique_id not in generated_ids:
            generated_ids.add(unique_id)  # Ajoute l'ID à l'ensemble des IDs générés
            list_rows.append(
                (unique_id, first_name, last_name, pwd, age, fake.job(), fake.address(), fake.phone_number(), email)
            )
    return list_rows

In [74]:
data = generate_data()

In [75]:
client = pd.DataFrame(data, columns=['id', 'prenom', 'nom', 'pwd', 'age', 'job', 'adress', 'phoneNumber', 'email'])
client.head()

,id,prenom,nom,pwd,age,job,adress,phoneNumber,email
0,433232,Shawn,Williams,Tl9&Z-Gkyj8U9n@$,24,"Teacher, English as a foreign language","PSC 0696, Box 2723\nAPO AA 26226",+1-531-674-4789,shawn.williams@jacobs.fr
1,974687,Andrew,Cole,Xz1_WD43tryO/}W!,21,Data processing manager,"PSC 5539, Box 5101\nAPO AP 23763",+1-411-583-8691x09137,andrew.cole@serrano.fr
2,887599,Brian,Brooks,Ie1#VG#0mW;7!JS:,18,Advertising copywriter,"427 Amy Radial\nNorth Jennifershire, VA 19585",(605)523-1350x471,brian.brooks@franklin.org
3,355750,Travis,Cuevas,Qk3(eE\K.P|s5YLf,48,Private music teacher,"2399 Kelsey Mission\nRiveraton, MI 92997",727.598.1085,travis.cuevas@sullivan.net
4,586574,Lisa,Allen,Dx1*wk[.:t}\91[D,18,Science writer,"21453 Caleb Vista Suite 238\nWest Johnhaven, F...",+1-209-311-4416,lisa.allen@chen.gov


In [8]:
client.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11890 entries, 0 to 11889
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           11890 non-null  int64 
 1   prenom       11890 non-null  object
 2   nom          11890 non-null  object
 3   pwd          11890 non-null  object
 4   age          11890 non-null  int64 
 5   job          11890 non-null  object
 6   adress       11890 non-null  object
 7   phoneNumber  11890 non-null  object
 8   email        11890 non-null  object
dtypes: int64(2), object(7)
memory usage: 836.1+ KB


In [77]:
client.to_csv('client.csv', index=False)

# Fake game stats data

In [16]:
# Générer les IDs de 2 à 50
all_ids = np.arange(2, 52)
excluded_ids = [8, 11, 12, 13, 14, 15, 16, 27, 28, 29, 34, 45]
id_values = [e for e in all_ids if e not in excluded_ids]

# Générer toutes les dates de 2023
start_date = pd.to_datetime('2022-01-01')
end_date = pd.to_datetime('2023-12-31')
dates = pd.date_range(start_date, end_date, freq='D')

np.random.seed(10)

# Créer une DataFrame vide pour stocker les données
jeux_stats = pd.DataFrame(columns=['id_article', 'date', 'qte_ajoutee', 'qte_achetee'])

# Générer des quantités aléatoires en fonction de chaque article et date
for article_id in id_values:
    # Générer des paramètres spécifiques pour chaque article (moyenne et écart-type)
    moyenne_ajoutee = np.random.uniform(50, 200)  # Moyenne pour qte_ajoutee
    ecart_type_ajoutee = np.random.uniform(20, 100)  # Écart-type pour qte_ajoutee
    
    moyenne_achetee = moyenne_ajoutee * np.random.uniform(0.5, 0.8)  # Moyenne pour qte_achetee
    ecart_type_achetee = ecart_type_ajoutee * np.random.uniform(0.5, 0.8)  # Écart-type pour qte_achetee
    
    # Générer des quantités aléatoires pour chaque date en utilisant une distribution normale
    qte_ajoutee = np.random.normal(moyenne_ajoutee, ecart_type_ajoutee, len(dates)).astype(int)
    qte_ajoutee = np.abs(qte_ajoutee)  # Rendre les valeurs négatives positives
    qte_achetee = np.random.normal(moyenne_achetee, ecart_type_achetee, len(dates)).astype(int)
    qte_achetee = np.abs(qte_achetee)  # Rendre les valeurs négatives positives
    qte_achetee = np.where(qte_achetee == 0, 1, qte_achetee) # Assurer que qte_achetee n'est pas nulle
    
    # Créer une DataFrame pour cet article et concaténer avec la DataFrame principale
    article_data = pd.DataFrame({
        'id_article': article_id,
        'date': dates,
        'qte_ajoutee': qte_ajoutee,
        'qte_achetee': qte_achetee
    })
    jeux_stats = pd.concat([jeux_stats, article_data])

# Réinitialiser l'index
jeux_stats.reset_index(drop=True, inplace=True)
jeux_stats.loc[(jeux_stats['date'].dt.year == 2023), 'qte_ajoutee'] += 180
jeux_stats.loc[(jeux_stats['date'].dt.year == 2023), 'qte_achetee'] += 50

# Convertir la colonne 'date' en format date YYYY-MM-DD
jeux_stats['date'] = jeux_stats['date'].dt.strftime('%Y-%m-%d')

C:\Users\PC de MEDUSA\AppData\Local\Temp\ipykernel_38836\65256003.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  jeux_stats = pd.concat([jeux_stats, article_data])


In [17]:
jeux_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27740 entries, 0 to 27739
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id_article   27740 non-null  object
 1   date         27740 non-null  object
 2   qte_ajoutee  27740 non-null  object
 3   qte_achetee  27740 non-null  object
dtypes: object(4)
memory usage: 867.0+ KB


In [18]:
jeux_stats.to_csv('jeux_stats.csv', index=False)